In [1]:
import cv2
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import filedialog
import cvlib
from cvlib.object_detection import draw_bbox
import os
from datetime import datetime

output_folder = "output_images"  # Folder to save captured images

def detect_speed(vidPath, speed_threshold):
    def detect_objects():
        nonlocal video_stream, prev_frame, object_detected, video_ended
        ret, frame = video_stream.read()

        if ret:
            frame = cv2.resize(frame, (600, 400))
            bbox, labels, conf = cvlib.detect_common_objects(frame)

            # Calculate displacement and speed of detected objects
            if prev_frame is not None:
                displacement = calculate_displacement(prev_frame, frame)
                fps = video_stream.get(cv2.CAP_PROP_FPS)
                speed = displacement * fps
                if speed > speed_threshold:
                    object_detected = True
                    print(f"Vehicle detected! Speed: {speed:.2f} pixels/second")

                    # Capture and save the image of the detected object
                    capture_and_save_image(frame)

            prev_frame = frame

            # Draw bounding boxes on the frame and display it in the Tkinter window
            frame = draw_bbox(frame, bbox, labels, conf)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            photo = ImageTk.PhotoImage(image=Image.fromarray(frame))
            label.config(image=photo)
            label.image = photo

            # Continue detecting objects if video is ongoing, else release resources
            if not video_ended:
                label.after(10, detect_objects)
            else:
                video_stream.release()
                root_video.destroy()  # Close the video window when video ends
        else:
            # Release resources and close window after 1 second if video ends
            video_stream.release()
            video_ended = True
            root_video.after(1000, lambda: root_video.destroy())

    def calculate_displacement(prev, curr):
        # Calculate optical flow displacement between two frames
        prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Calculate the average magnitude of optical flow vectors
        magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        return magnitude.mean()

    def capture_and_save_image(frame):
        # Create output folder if it doesn't exist
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Get current date and time for timestamp
        now = datetime.now()
        timestamp = now.strftime("%Y%m%d_%H%M%S")

        # Save the captured image with timestamp as filename
        img_name = f"{output_folder}/detected_object_{timestamp}.jpg"
        cv2.imwrite(img_name, frame)
        print(f"Image saved: {img_name}")

    # Initialize tkinter window for video display
    root_video = tk.Toplevel()
    root_video.title("Object and Speed Detection")
    root_video.geometry("640x480")

    # Create a label to display the video feed
    label = tk.Label(root_video)
    label.pack(padx=10, pady=10)

    # Open selected video file or access the camera
    if vidPath.isdigit():  # If vidPath is a digit, consider it as the camera index
        video_stream = cv2.VideoCapture(int(vidPath))
    else:
        video_stream = cv2.VideoCapture(vidPath)

    prev_frame = None
    object_detected = False
    video_ended = False

    # Start object detection and speed estimation
    detect_objects()

    # Function to release the camera when the window is closed
    def close_window():
        nonlocal video_stream
        video_stream.release()
        root_video.destroy()

    root_video.protocol("WM_DELETE_WINDOW", close_window)

def open_and_detect_speed():
    # Open file dialog to select a video file and start speed detection
    file_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4;*.avi;*.mkv")])
    if file_path:
        detect_speed(file_path, speed_threshold=20)  # Modify the speed threshold as needed

def toggle_camera():
    # Toggle between using the camera and uploading a video file
    if camera_button['text'] == "Use Camera":
        camera_button['text'] = "Use Video"
        upload_button.config(state=tk.DISABLED)
        detect_speed("0", speed_threshold=50)  # "0" represents the default camera index
    else:
        camera_button['text'] = "Use Camera"
        upload_button.config(state=tk.NORMAL)

# Initialize tkinter window
root = tk.Tk()
root.title("Vehicle Speed Detection Assignment")
root.geometry("300x150")

# Button to upload video and detect speed
upload_button = tk.Button(root, text="Upload Video", command=open_and_detect_speed)
upload_button.pack(padx=20, pady=10)

# Button to toggle camera
camera_button = tk.Button(root, text="Use Camera", command=toggle_camera)
camera_button.pack(padx=20, pady=10)

# Run the tkinter main loop
root.mainloop()


Vehicle detected! Speed: 24.48 pixels/second
Image saved: output_images/detected_object_20240128_215335.jpg
Vehicle detected! Speed: 22.11 pixels/second
Image saved: output_images/detected_object_20240128_215336.jpg
Vehicle detected! Speed: 23.25 pixels/second
Image saved: output_images/detected_object_20240128_215337.jpg
Vehicle detected! Speed: 29.38 pixels/second
Image saved: output_images/detected_object_20240128_215338.jpg
Vehicle detected! Speed: 27.46 pixels/second
Image saved: output_images/detected_object_20240128_215340.jpg
Vehicle detected! Speed: 24.49 pixels/second
Image saved: output_images/detected_object_20240128_215341.jpg
Vehicle detected! Speed: 27.03 pixels/second
Image saved: output_images/detected_object_20240128_215342.jpg
Vehicle detected! Speed: 21.50 pixels/second
Image saved: output_images/detected_object_20240128_215344.jpg
Vehicle detected! Speed: 22.45 pixels/second
Image saved: output_images/detected_object_20240128_215346.jpg
Vehicle detected! Speed: 22.